<a href="https://colab.research.google.com/github/jaleel-muhiyadeen-206/AI-Powered-Intelligent-Tourism-Support-System/blob/main/CNN_Model_Implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
!pip install -q opencv-python tensorflow
import os, cv2, numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from google.colab import drive
import tensorflow as tf
from tensorflow.keras import layers, Model
from tensorflow.keras.callbacks import EarlyStopping

In [20]:
drive.mount('/content/drive', force_remount=True)
DATA_FOLDER = '/content/drive/MyDrive/LandMark Images'

Mounted at /content/drive


In [21]:
# Loading the image data
def load_images():
    X, y = [], []
    classes = sorted([d for d in os.listdir(DATA_FOLDER) if os.path.isdir(os.path.join(DATA_FOLDER, d))])

    for i, name in enumerate(classes):
        path = os.path.join(DATA_FOLDER, name)
        for file in os.listdir(path):
            if file.lower().endswith(('.jpg', '.jpeg', '.webp', '.JPG')):
                try:
                    img = np.array(Image.open(os.path.join(path, file)).convert('RGB'))
                    X.append(cv2.resize(img, (350, 350)) / 255.0)
                    y.append(i)
                except:
                    pass

    idx = np.random.permutation(len(X))
    split = int(0.8 * len(X))
    return np.array(X)[idx[:split]], np.array(X)[idx[split:]], np.array(y)[idx[:split]], np.array(y)[idx[split:]], classes

In [22]:
X_train, X_val, y_train, y_val, CLASS_NAMES = load_images()
print(f"Loaded {len(X_train)} train, {len(X_val)} val images")

Loaded 172 train, 44 val images


In [23]:
# Creating the CNN Model
base = tf.keras.applications.MobileNetV2(input_shape=(350, 350, 3), include_top=False, weights='imagenet')
base.trainable = False

inputs = layers.Input(shape=(350, 350, 3))
x = base(inputs)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.3)(x)
outputs = layers.Dense(len(CLASS_NAMES), activation='softmax')(x)

model = Model(inputs, outputs)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()

/tmp/ipython-input-3464995674.py:2: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base = tf.keras.applications.MobileNetV2(input_shape=(350, 350, 3), include_top=False, weights='imagenet')


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_3 (InputLayer)      │ (None, 350, 350, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ mobilenetv2_1.00_224            │ (None, 11, 11, 1280)   │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_1      │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 1280)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 24)             │        30,744 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,288,728 (8.73 MB)

 Trainable params: 30,744 (120.09 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

In [24]:
# Model Training process
model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=20, batch_size=9,
          callbacks=[tf.keras.callbacks.EarlyStopping(patience=4)], verbose=1)

Epoch 1/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 36s 2s/step - accuracy: 0.0249 - loss: 3.4574 - val_accuracy: 0.1364 - val_loss: 3.0262
Epoch 2/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 21s 1s/step - accuracy: 0.3011 - loss: 2.4111 - val_accuracy: 0.2955 - val_loss: 2.4068
Epoch 3/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 20s 1s/step - accuracy: 0.7077 - loss: 1.7061 - val_accuracy: 0.5682 - val_loss: 2.0220
Epoch 4/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 18s 922ms/step - accuracy: 0.8014 - loss: 1.2407 - val_accuracy: 0.6591 - val_loss: 1.7104
Epoch 5/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 20s 1s/step - accuracy: 0.8958 - loss: 0.9376 - val_accuracy: 0.6364 - val_loss: 1.5817
Epoch 6/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 19s 958ms/step - accuracy: 0.9624 - loss: 0.6068 - val_accuracy: 0.6818 - val_loss: 1.4530
Epoch 7/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 19s 913ms/step - accuracy: 0.9550 - loss: 0.5743 - val_accuracy: 0.6818 - val_loss: 1.4771
Epoch 8/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 20s 1s/step - accuracy: 0.9611 - loss: 0.4137 - val_accuracy: 0.6818 - 

In [29]:
# Locations of landmarks from my image dataset
LOCATION_MAP = {
    "Adam's Peak": "Rathnapura, Sabaragamuwa Province, Sri Lanka",
    "Ancient City of Polonnaruwa": "Polonnaruwa, North Central Province, Sri Lanka",
    "Beruwala Light House": "Beruwala, Western Province, Sri Lanka",
    "British War Cemetery": "Kandy, Central Province, Sri Lanka",
    "Bundala National Park": "Hambantota, Southern Province, Sri Lanka",
    "Delft Island": "Jaffna, Northern Province, Sri Lanka",
    "Dowa Rock Temple": "Bandarawela, Uva Province, Sri Lanka",
    "Ganagaramaya Temple": "Colombo, Western Province, Sri Lanka",
    "Henarathgoda Botanical Gard": "Gampaha, Western Province, Sri Lanka",
    "Hortains Plain": "Nuwara Eliya, Central Province, Sri Lanka",
    "Independance Square": "Colombo, Western Province, Sri Lanka",
    "Jaya Sri Maha Bodhi": "Anuradhapura, North Central Province, Sri Lanka",
    "Lotus Tower": "Colombo, Western Province, Sri Lanka",
    "Maligawa Buddha Statue": "Kandy, Central Province, Sri Lanka",
    "Nine Arches Bridge": "Ella, Uva Province, Sri Lanka",
    "Pinnawala Elephant Orphanage": "Kegalle, Sabaragamuwa Province, Sri Lanka",
    "Sigiriya": "Dambulla, Central Province, Sri Lanka",
    "Sinharaja Forest": "Ratnapura, Sabaragamuwa Province, Sri Lanka",
    "Sri Dalada Maligawa": "Kandy, Central Province, Sri Lanka",
    "Star Fort": "Matara, Southern Province, Sri Lanka",
    "Turtle Hatchery": "Kosgoda, Southern Province, Sri Lanka",
    "Vavuniya Archaeological Museum": "Vavuniya, Northern Province, Sri Lanka",
    "Wilapattu National Park": "Puttalam, North Western Province, Sri Lanka",
    "Yapahuwa Rock Fortress": "Yapahuwa, North Western Province, Sri Lanka",
}

In [33]:
# This function returns the landmark name and its location
def predict(img_path):
    """Returns a dict with 'name' and 'place' keys"""
    img = Image.open(img_path).convert('RGB')
    img_array = cv2.resize(np.array(img), (224, 224)) / 255.0
    img_array = np.expand_dims(img_array, axis=0)

    pred_idx = np.argmax(model.predict(img_array, verbose=0)[0])
    landmark = CLASS_NAMES[pred_idx]
    location = LOCATION_MAP.get(landmark, "Unknown")

    return {'name': landmark, 'place': location}

In [35]:
# Testing the model
result = predict('/content/drive/MyDrive/LandMark Images/Star Fort/pic 1.jpg')
print(f"Landmark: {result['name']}")
print(f"Location: {result['place']}")


Landmark: Star Fort
Location: Matara, Southern Province, Sri Lanka
